# [Bassat et al., 2009](https://pubmed.ncbi.nlm.nih.gov/19936217/) and [Baraka et al., 2014](https://journals.asm.org/doi/10.1128/aac.03647-14)

### **Drug:** AL
### **Genotype:** pfcrt K76 (WT)
### **Number of patients in arm:** 27
### **Number of patients cured:** 25
### **Efficacy:** 0.9259 or 92.59%

### *Loading required libraries*

In [1]:
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from timeit import default_timer as timer
from matplotlib.ticker import MaxNLocator
import pickle

## Simulating 10k patients with default values

In [4]:
command_default = ["/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk", "--AL", "-n", "1000"]
output_default = subprocess.run(command_default, capture_output=True, text=True)
lines_default = output_default.stdout.splitlines()
df_default = pd.DataFrame([line.split(',') for line in lines_default], columns=["PID","HOUR","COMP2CONC_ART","COMP2CONC_LUM","PARASITEDENSITY"])
df_default = df_default.iloc[1:]
df_default = df_default.apply(pd.to_numeric, errors='coerce')
    

In [7]:
#Saving the main dataframe to a pickle file
#pickle.dump(df_default, open('pkpd_AL_bassat_baraka_pfcrt.K76_default.pyobj', 'wb')) 
#df_default.to_csv('pkpd_AL_bassat_baraka_pfcrt.K76_default.csv', index=False, quoting= False)

In [4]:
#Loading the main dataframe from a pickle file
df_default = pickle.load(open('pkpd_AL_bassat_baraka_pfcrt.K76_default.pyobj', 'rb'))

In [21]:
#Creating new column with log(parasite density), adding 0.00001 to avoid log(0) error
df_default['LOG_PARASITEDENSITY'] = np.log10(df_default['PARASITEDENSITY'] + 0.00001)

df_default.head()

,PID,HOUR,COMP2CONC_ART,COMP2CONC_LUM,PARASITEDENSITY,LOG_PARASITEDENSITY
1,0,0.0,0.000,0.000,20000.000,4.301030
2,0,1.0,30.012,1014.819,8522.690,3.930577
3,0,2.0,17.599,1875.401,4426.372,3.646048
4,0,3.0,10.223,2600.930,2541.210,3.405041
5,0,4.0,5.938,3208.326,1531.515,3.185121


In [30]:
#Getting number of patients with treatment failure
df_default_day28_failed = df_default[(df_default['PARASITEDENSITY'] >= 10) & (df_default['HOUR'] == 671.0)]
df_default_day28_failed_pids = df_default_day28_failed['PID'].unique().tolist()
df_default_day28_failed_pids = np.array(df_default_day28_failed_pids)
print(df_default_day28_failed['PID'].unique().tolist())
len(df_default_day28_failed['PID'].unique().tolist())


[]


0

# Simulating 95% efficacy in 1000 patients by only adjusting pmax_art

### Default pmax_art: 0.99997

### 1% decrease: 0.99997 - (0.99997 * 0.01) = 0.9899703
### **2.5% decrease: 0.99997 - (0.99997 * 0.025) = 0.97497075 gives ~95% efficacy**
### 5% decrease: 0.99997 - (0.99997 * 0.05) = 0.9499715
### 10% decrease: 0.99997 - (0.99997 * 0.1) = 0.899973
### 25% decrease: 0.99997 - (0.99997 * 0.25) = 0.7499775
### 50% decrease: 0.99997 - (0.99997 * 0.5) = 0.499985
### 75% decrease: 0.99997 - (0.99997 * 0.75) = 0.24999250000000006


In [128]:
command_pmax_art_adj = ["/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk", "--AL", "-n", "1000", "--pmax_art", "0.97497075"]
output_pmax_art_adj = subprocess.run(command_pmax_art_adj, capture_output=True, text=True)
lines_pmax_art_adj = output_pmax_art_adj.stdout.splitlines()
df_pmax_art_adj = pd.DataFrame([line.split(',') for line in lines_pmax_art_adj], columns=["PID","HOUR","COMP2CONC_ART","COMP2CONC_LUM","PARASITEDENSITY"])
df_pmax_art_adj = df_pmax_art_adj.iloc[1:]
df_pmax_art_adj = df_pmax_art_adj.apply(pd.to_numeric, errors='coerce')
    

In [135]:
#Saving the main dataframe to a pickle file
pickle.dump(df_pmax_art_adj, open('pkpd_AL_bassat_baraka_pfcrt.K76_pmax_art_adj.pyobj', 'wb')) 
df_pmax_art_adj.to_csv('pkpd_AL_bassat_baraka_pfcrt.K76_pmax_art_adj.csv', index=False, quoting= False)

In [120]:
#Loading the main dataframe from a pickle file
df_pmax_art_adj = pickle.load(open('pkpd_AL_bassat_baraka_pfcrt.K76_pmax_art_adj.pyobj', 'rb'))

In [129]:
#Creating new column with log(parasite density), adding 0.00001 to avoid log(0) error
df_pmax_art_adj['LOG_PARASITEDENSITY'] = np.log10(df_pmax_art_adj['PARASITEDENSITY'] + 0.00001)

df_pmax_art_adj.head()

,PID,HOUR,COMP2CONC_ART,COMP2CONC_LUM,PARASITEDENSITY,LOG_PARASITEDENSITY
1,0,0.0,0.000,0.000,20000.000,4.301030
2,0,1.0,2.626,1242.087,10516.002,4.021851
3,0,2.0,18.868,2263.636,6558.456,3.816802
4,0,3.0,19.679,3097.426,4558.063,3.658780
5,0,4.0,12.590,3771.555,3360.425,3.526394


In [130]:
#Getting number of patients with treatment failure
df_pmax_art_adj_day28_failed = df_pmax_art_adj[(df_pmax_art_adj['PARASITEDENSITY'] >= 10) & (df_pmax_art_adj['HOUR'] == 671.0)]
df_pmax_art_adj_day28_failed_pids = df_pmax_art_adj_day28_failed['PID'].unique().tolist()
df_pmax_art_adj_day28_failed_pids = np.array(df_pmax_art_adj_day28_failed_pids)
print(df_pmax_art_adj_day28_failed['PID'].unique().tolist())
len(df_pmax_art_adj_day28_failed['PID'].unique().tolist())


[12, 62, 69, 98, 112, 169, 183, 194, 200, 247, 256, 267, 318, 432, 464, 481, 482, 486, 544, 556, 560, 561, 578, 595, 622, 630, 638, 643, 663, 684, 689, 694, 701, 712, 736, 828, 840, 857, 903, 927, 942, 945, 971, 996]


44

# Simulating 92.59% efficacy in 1000 patients by adjusting both ec50_art and pmax_art

### Default pmax_art: 0.99997
### 2.5% decrease: 0.99997 - (0.99997 * 0.025) = 0.97497075

### Default ec50_art: 0.1
### **10% increase: 0.1 + (0.1 * 0.1) = 0.11000000000000001 gives exactly 95% efficacy**
### **20% increase: 0.1 + (0.1 * 0.2) = 0.12000000000000001 gives ~95% efficacy**

In [152]:
0.1 + (0.1 * 0.2)

0.12000000000000001

In [157]:
command_ec50_pmax_art_adj = ["/home/venitha_b/Projects/2019-test-ppq-pk/build/run_ppq_pk", "--AL", "-n", "1000", "--pmax_art", "0.97497075", "--ec50_art", "0.12000000000000001"]
output_ec50_pmax_art_adj = subprocess.run(command_ec50_pmax_art_adj, capture_output=True, text=True)
lines_ec50_pmax_art_adj = output_ec50_pmax_art_adj.stdout.splitlines()
df_ec50_pmax_art_adj = pd.DataFrame([line.split(',') for line in lines_ec50_pmax_art_adj], columns=["PID","HOUR","COMP2CONC_ART","COMP2CONC_LUM","PARASITEDENSITY"])
df_ec50_pmax_art_adj = df_ec50_pmax_art_adj.iloc[1:]
df_ec50_pmax_art_adj = df_ec50_pmax_art_adj.apply(pd.to_numeric, errors='coerce')
    

In [134]:
#Saving the main dataframe to a pickle file
pickle.dump(df_ec50_pmax_art_adj, open('pkpd_AL_bassat_baraka_pfcrt.K76_ec50_pmax_art_adj.pyobj', 'wb')) 
df_ec50_pmax_art_adj.to_csv('pkpd_AL_bassat_baraka_pfcrt.K76_ec50_pmax_art_adj.csv', index=False, quoting= False)

In [ ]:
#Loading the main dataframe from a pickle file
df_ec50_pmax_art_adj = pickle.load(open('pkpd_AL_bassat_baraka_pfcrt.K76_ec50_pmax_art_adj.pyobj', 'rb'))

In [158]:
#Creating new column with log(parasite density), adding 0.00001 to avoid log(0) error
df_ec50_pmax_art_adj['LOG_PARASITEDENSITY'] = np.log10(df_ec50_pmax_art_adj['PARASITEDENSITY'] + 0.00001)

df_ec50_pmax_art_adj.head()

,PID,HOUR,COMP2CONC_ART,COMP2CONC_LUM,PARASITEDENSITY,LOG_PARASITEDENSITY
1,0,0.0,0.000,0.000,20000.000,4.301030
2,0,1.0,28.793,416.615,10195.616,4.008413
3,0,2.0,18.188,797.941,6411.922,3.806988
4,0,3.0,9.912,1146.250,4474.551,3.650749
5,0,4.0,5.400,1463.682,3284.722,3.516499


In [159]:
#Getting number of patients with treatment failure
df_ec50_pmax_art_adj_day28_failed = df_ec50_pmax_art_adj[(df_ec50_pmax_art_adj['PARASITEDENSITY'] >= 10) & (df_ec50_pmax_art_adj['HOUR'] == 671.0)]
df_ec50_pmax_art_adj_day28_failed_pids = df_ec50_pmax_art_adj_day28_failed['PID'].unique().tolist()
df_ec50_pmax_art_adj_day28_failed_pids = np.array(df_ec50_pmax_art_adj_day28_failed_pids)
print(df_ec50_pmax_art_adj_day28_failed['PID'].unique().tolist())
len(df_ec50_pmax_art_adj_day28_failed['PID'].unique().tolist())


[2, 13, 81, 88, 102, 144, 170, 182, 197, 203, 238, 246, 255, 270, 306, 315, 497, 533, 534, 591, 626, 634, 683, 721, 744, 761, 784, 789, 816, 819, 833, 877, 899, 902, 908, 918, 919, 924, 925, 959, 975, 980, 982]


43

In [160]:
100 - ((43/1000)* 100)

95.7